# Análise e Categorização de Clusters para Agricultura e Floresta

Este notebook guia o processo de análise cuidadosa e categorização manual dos clusters identificados pelo modelo não supervisionado, com foco específico em áreas agrícolas e florestais.

## Objetivos:
1. Analisar características espectrais e espaciais de cada cluster
2. Validar interpretações com dados de referência
3. Documentar o processo de decisão
4. Atribuir categorias com níveis de confiança apropriados

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from src.models.cluster_categorizer import ClusterCategorizer
from src.models.unsupervised_cnn import UnsupervisedCNN
from src.data.tiff_loader import TiffLoader

%matplotlib inline
plt.style.use('seaborn')

## 1. Carregar Modelo e Dados

Primeiro, vamos carregar o modelo treinado e alguns dados para análise:

In [ ]:
# Carregar modelo e dados
model = UnsupervisedCNN.load_model('path_to_your_model')
loader = TiffLoader(train_val_dir='../data/train')
train_ds, _ = loader.load_train_val_data()
X_samples = np.concatenate([batch.numpy() for batch in train_ds.take(20)], axis=0)
predictions = model.predict(X_samples)

## 2. Análise Espectral dos Clusters

Vamos analisar as características espectrais de cada cluster:

In [ ]:
def plot_spectral_signatures(X, predictions, n_clusters):
    """Plotar assinaturas espectrais médias por cluster"""
    plt.figure(figsize=(12, 6))
    bands = ['NIR (B8)', 'Red (B4)', 'SWIR (B11)']
    
    for cluster in range(n_clusters):
        cluster_samples = X[predictions == cluster]
        mean_signature = np.mean(cluster_samples, axis=(0,1))  # Média por banda
        plt.plot(bands, mean_signature, 'o-', label=f'Cluster {cluster}')
    
    plt.title('Assinaturas Espectrais por Cluster')
    plt.xlabel('Bandas')
    plt.ylabel('Reflectância Média')
    plt.legend()
    plt.grid(True)

plot_spectral_signatures(X_samples, predictions, model.n_clusters)

## 3. Análise de Textura e Padrões Espaciais

Vamos examinar as características espaciais de cada cluster:

In [ ]:
def analyze_spatial_patterns(X, predictions, cluster_id, n_samples=10):
    """Análise detalhada de padrões espaciais para um cluster"""
    cluster_samples = X[predictions == cluster_id][:n_samples]
    
    plt.figure(figsize=(15, 6))
    for i, sample in enumerate(cluster_samples):
        plt.subplot(2, 5, i+1)
        plt.imshow(sample)
        plt.axis('off')
        
    plt.suptitle(f'Análise de Padrões Espaciais - Cluster {cluster_id}')
    plt.tight_layout()

# Analisar cada cluster individualmente
for cluster in range(model.n_clusters):
    analyze_spatial_patterns(X_samples, predictions, cluster)

## 4. Checklist de Validação

Para cada cluster, vamos preencher um checklist de características antes de atribuir uma categoria:

In [ ]:
def create_cluster_checklist(cluster_id):
    """Criar um checklist interativo para análise do cluster"""
    checklist = pd.DataFrame({
        'Característica': [
            'Alta reflectância no NIR',
            'Baixa reflectância no vermelho',
            'Padrões geométricos regulares',
            'Textura rugosa/irregular',
            'Bordas bem definidas',
            'Variação interna homogênea',
            'Indícios de atividade agrícola',
            'Características de floresta'
        ],
        'Presente': ['Não' for _ in range(8)],
        'Confiança': [0.0 for _ in range(8)]
    })
    
    return checklist

# Criar checklist para cada cluster
cluster_checklists = {}
for cluster in range(model.n_clusters):
    cluster_checklists[cluster] = create_cluster_checklist(cluster)
    print(f"\nCluster {cluster}:")
    display(cluster_checklists[cluster])

## 5. Análise Temporal (se disponível)

Se você tiver dados temporais disponíveis, é importante analisar as mudanças ao longo do tempo:

In [ ]:
def plot_temporal_patterns(cluster_id):
    """Plotar padrões temporais se disponíveis"""
    # Este é um exemplo - implemente de acordo com seus dados temporais
    print(f"Análise temporal para Cluster {cluster_id}:")
    print("- Verifique variações sazonais")
    print("- Compare com calendário agrícola da região")
    print("- Observe padrões de manejo florestal")

## 6. Processo de Categorização

Agora vamos categorizar cada cluster baseado nas evidências coletadas:

In [ ]:
categorizer = ClusterCategorizer(n_clusters=model.n_clusters)

def categorize_cluster_with_evidence(cluster_id, checklist):
    """Categorizar cluster com base nas evidências"""
    # Análise das características predominantes
    agricultural_score = 0
    forest_score = 0
    
    # Calcular scores baseado no checklist
    # Implemente sua lógica de pontuação aqui
    
    print(f"\nAnálise do Cluster {cluster_id}:")
    print(f"Score Agrícola: {agricultural_score}")
    print(f"Score Florestal: {forest_score}")
    
    # Solicitar input do usuário para confirmação
    print("\nCategorias possíveis:")
    print("1. Floresta densa")
    print("2. Agricultura estabelecida")
    print("3. Agricultura inicial")
    print("4. Floresta em regeneração")
    print("5. Sistema agroflorestal")
    print("6. Outro (especificar)")
    
    # Aqui você deve interagir com o usuário para confirmar a categoria
    print("\nPor favor, revise as evidências e confirme a categoria manualmente.")

# Analisar cada cluster
for cluster in range(model.n_clusters):
    categorize_cluster_with_evidence(cluster, cluster_checklists[cluster])

## 7. Documentação e Validação Final

Para cada cluster categorizado, documente:

1. **Evidências Utilizadas**:
   - Características espectrais
   - Padrões espaciais
   - Dados temporais (se disponíveis)
   - Conhecimento local

2. **Nível de Confiança**:
   - Baseado na quantidade e qualidade das evidências
   - Considerar ambiguidades e incertezas

3. **Validação Cruzada**:
   - Comparação com mapas existentes
   - Verificação em campo (se possível)
   - Opinião de especialistas

4. **Limitações**:
   - Documentar casos ambíguos
   - Identificar possíveis fontes de erro
   - Sugerir melhorias para futuras análises

In [ ]:
def document_categorization(cluster_id, category, confidence, evidence):
    """Documentar a categorização final com todas as evidências"""
    categorizer.categorize_cluster(
        cluster_id=cluster_id,
        category=category,
        description=evidence,
        confidence=confidence
    )

# Exemplo de documentação
evidence_example = """
Evidências para categorização:
1. Características espectrais:
   - Alta reflectância no NIR
   - Baixa reflectância no vermelho
2. Padrões espaciais:
   - Textura homogênea
   - Ausência de padrões geométricos
3. Validação:
   - Confirmado com imagens de alta resolução
   - Consistente com mapa florestal da região
4. Limitações:
   - Possível confusão em áreas de borda
   - Necessidade de validação sazonal
"""

# Salvar a documentação final
categorizer.save_categories('../data/categorias_validadas.json')